# **MULTI-LAYER PERCEPTRON IMPLEMENTATION FOR CIFAR-10**

*Spyrakis Angelos, ECE AUTh (9352)*


---
Table of Contents:

**1. Fully-connected MLP**

> 1.1. Fine tuning with KerasTuner
>
> 1.2. Fine tuning with KerasTuner (+ Dropout)
>
> 1.3. Fine-tuned model


**2. Convolutional Neural Network**

> 2.1.   CNN Implementation (RGB)
>
> 2.2. CNN Implementation (Grayscale)

---
Execute the two following cells to download the dataset and the tuner used.

In [ ]:
!wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O - | tar -xz

In [ ]:
pip install -q -U keras-tuner

# 1. Fully-connected MLP


## 1.1. Fine-tuning using a keras tuner

In [ ]:
"""
Solving the CIFAR-10 classification task with a
Multi-Layer Perceptron NN structure.

Fine-tuning using the Random Search keras tuner.
"""
import matplotlib.pyplot as plt
from keras import backend as K
import keras_tuner as kt
import tensorflow as tf
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential


def unpickle(file):
    """
    Unpickles files produced with cPickle
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_cifar10():
    """
    Loads the CIFAR-10 dataset

    Output:
    X_train -> training images (RGB)
    y_train -> training images' labels
    X_test -> testing images (RGB)
    y_test -> testing images' labels
    """
    dict = unpickle("cifar-10-batches-py/data_batch_1")
    X_train = dict[b'data']
    y_train = np.array(dict[b'labels'], dtype=int)

    dict = unpickle("cifar-10-batches-py/data_batch_2")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_3")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_4")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_5")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/test_batch")
    X_test = dict[b'data']
    y_test = np.array(dict[b'labels'], dtype=int)

    # Normalize data
    X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0

    # One-hot encode output, because allowing the model to assume a natural ordering 
    # between categories may result in poor performance or unexpected results
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)  #fit_transform learns the mean and variance of the data
    y_test_original = y_test  #maintain original labels
    y_test = lb.transform(y_test)  #tansform uses the same mean and var as above

    return X_train, y_train, X_test, y_test


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_metric(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def build_model(hp):
    num_of_classes = 10
    input_shape = (32*32*3,)

    # Create the model
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()

    hp_units_first = hp.Choice('units_first', values=[2048, 1024, 512, 256])
    model.add(Dense(hp_units_first, input_shape=input_shape, activation='relu', kernel_initializer=initializer))

    hp_units_second = hp.Choice('units_second', values=[2048, 1024, 512, 256])
    model.add(Dense(hp_units_second, activation='relu', kernel_initializer=initializer))

    model.add(Dropout(0.3))

    model.add(Dense(num_of_classes, activation='softmax'))

    # Configure the model and start training
    hp_lr = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001])
    opt = tf.keras.optimizers.Adam(learning_rate = hp_lr)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.CategoricalAccuracy(), f1_metric])

    return model


def main():
    # Load and process data
    X_train, y_train, X_test, y_test = load_cifar10()

    # Set up tuner
    tuner = kt.RandomSearch(build_model, objective=kt.Objective('val_f1_metric', direction='max'), max_trials= 48, executions_per_trial=4, overwrite=True)

    start_time = time.time()

    tuner.search(X_train, y_train, epochs=60, batch_size=1000, validation_split=0.2)

    elapsed_time = time.time() - start_time
    print(f'Tuning time = {elapsed_time} second(s)')

    best_hps=tuner.get_best_hyperparameters(1)[0]
    
    print(f"""
    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is {best_hps.get('units_first')}, in the second densely-connected layer is {best_hps.get('units_second')} and the optimal learning rate 
    for the optimizer is {best_hps.get('learning_rate')}.
    """)


if __name__ == "__main__":
    main()

## 1.2. Fine-tuning using a keras tuner (+ Dropout)

In [ ]:
"""
Solving the CIFAR-10 classification task with a
Multi-Layer Perceptron NN structure.

Fine-tuning using the Random Search keras tuner (with Dropout).
"""
import matplotlib.pyplot as plt
from keras import backend as K
import keras_tuner as kt
import tensorflow as tf
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential


def unpickle(file):
    """
    Unpickles files produced with cPickle
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_cifar10():
    """
    Loads the CIFAR-10 dataset

    Output:
    X_train -> training images (RGB)
    y_train -> training images' labels
    X_test -> testing images (RGB)
    y_test -> testing images' labels
    """
    dict = unpickle("cifar-10-batches-py/data_batch_1")
    X_train = dict[b'data']
    y_train = np.array(dict[b'labels'], dtype=int)

    dict = unpickle("cifar-10-batches-py/data_batch_2")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_3")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_4")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_5")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/test_batch")
    X_test = dict[b'data']
    y_test = np.array(dict[b'labels'], dtype=int)

    # Normalize data
    X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0

    # One-hot encode output, because allowing the model to assume a natural ordering 
    # between categories may result in poor performance or unexpected results
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)  #fit_transform learns the mean and variance of the data
    y_test_original = y_test  #maintain original labels
    y_test = lb.transform(y_test)  #tansform uses the same mean and var as above

    return X_train, y_train, X_test, y_test


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_metric(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def build_model(hp):
    num_of_classes = 10
    input_shape = (32*32*3,)

    # Create the model
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()

    hp_units_first = hp.Choice('units_first', values=[2048, 1024, 512, 256])
    model.add(Dense(hp_units_first, input_shape=input_shape, activation='relu', kernel_initializer=initializer))

    hp_units_second = hp.Choice('units_second', values=[2048, 1024, 512, 256])
    model.add(Dense(hp_units_second, activation='relu', kernel_initializer=initializer))

    hp_dropout_prob = hp.Choice('dropout_prob', values=[0.25, 0.3, 0.4])
    model.add(Dropout(hp_dropout_prob))

    model.add(Dense(num_of_classes, activation='softmax'))

    # Configure the model and start training
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.CategoricalAccuracy(), f1_metric])

    return model


def main():
    # Load and process data
    X_train, y_train, X_test, y_test = load_cifar10()

    # Set up tuner
    tuner = kt.RandomSearch(build_model, objective=kt.Objective('val_f1_metric', direction='max'), max_trials= 48, executions_per_trial=4, overwrite=True)

    start_time = time.time()

    tuner.search(X_train, y_train, epochs=60, batch_size=1000, validation_split=0.2)

    elapsed_time = time.time() - start_time
    print(f'Tuning time = {elapsed_time} second(s)')

    best_hps=tuner.get_best_hyperparameters(1)[0]
    
    print(f"""
    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is {best_hps.get('units_first')}, in the second densely-connected layer is {best_hps.get('units_second')} and the optimal dropout probability 
    is {best_hps.get('dropout_prob')}.
    """)


if __name__ == "__main__":
    main()

## 1.3. Fine-tuned model

In [ ]:
"""
Solving the CIFAR-10 classification task with a
Multi-Layer Perceptron NN structure.

Fine-Tuned Network
"""
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix


def unpickle(file):
    """
    Unpickles files produced with cPickle
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_cifar10():
    """
    Loads the CIFAR-10 dataset

    Output:
    X_train -> training images (RGB)
    y_train -> training images' labels
    X_test -> testing images (RGB)
    y_test -> testing images' labels
    """
    dict = unpickle("cifar-10-batches-py/data_batch_1")
    X_train = dict[b'data']
    y_train = np.array(dict[b'labels'], dtype=int)

    dict = unpickle("cifar-10-batches-py/data_batch_2")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_3")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_4")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_5")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/test_batch")
    X_test = dict[b'data']
    y_test = np.array(dict[b'labels'], dtype=int)

    # Normalize data
    X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0

    # One-hot encode output, because allowing the model to assume a natural ordering 
    # between categories may result in poor performance or unexpected results
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)  #fit_transform learns the mean and variance of the data
    y_test_original = y_test  #maintain original labels
    y_test = lb.transform(y_test)  #tansform uses the same mean and var as above

    return X_train, y_train, X_test, y_test, y_test_original


def plot_model(history):
    # Plot accuracy
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title(f'Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_metric(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def build_model():
    num_of_classes = 10
    input_shape = (32*32*3,)

    # Create the model
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()

    model.add(Dense(2048, input_shape=input_shape, activation='relu', kernel_initializer=initializer))
    model.add(Dense(1024, activation='relu', kernel_initializer=initializer))
    model.add(Dropout(0.25))
    model.add(Dense(num_of_classes, activation='softmax'))

    # Configure the model and start training
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.CategoricalAccuracy(), f1_metric])

    return model


def print_examples(y_test, y_pred, X_test_flattened, labels):
    X_test = X_test_flattened.reshape(10000, 3, 32, 32).transpose(0,2,3,1)

    print('Printing examples of valid classification:')
    printed = 0
    fig = plt.figure(figsize=(4,4))
    for i in range (0, y_test.shape[0]):
        if y_test[i] == y_pred[i]:
            ax = fig.add_subplot(2, 2, printed + 1, xticks=[], yticks=[])
            ax.title.set_text(labels[y_pred[i]])
            ax.imshow((X_test[i, :]*255).astype(int))
            printed += 1
        if printed == 4:
            printed = 0
            break
    plt.show()
    print('Printing examples of false classification:')
    fig2 = plt.figure(figsize=(4,4))
    for i in range (0, y_test.shape[0]):
        if y_test[i] !=  y_pred[i]:
            ax2 = fig2.add_subplot(2, 2, printed + 1, xticks=[], yticks=[])
            ax2.title.set_text(labels[y_pred[i]])
            ax2.imshow((X_test[i, :]*255).astype(int))
            printed += 1
        if printed == 4:
            break
    plt.show()


def main():
    X_train, y_train, X_test, y_test, y_test_original = load_cifar10()
    model = build_model()
    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=60, batch_size=1000, verbose=1, validation_split=0.2)

    elapsed_time = time.time() - start_time
    print(f'Training time = {elapsed_time} second(s)')

    plot_model(history)

    # Test the model after training
    start_time = time.time()

    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    
    elapsed_time = time.time() - start_time
    print(f'Testing time = {elapsed_time} second(s)')

    # Plot confusion matrix
    prediction = model.predict(X_test) 
    y_pred = np.argmax(prediction, axis=1) #get predicted class
    labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    cm = confusion_matrix(y_test_original, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    fig, ax = plt.subplots(figsize=(15, 15))
    disp.plot(cmap=plt.cm.Blues, ax=ax)
    plt.show()

    print_examples(y_test_original, y_pred, X_test, labels)


if __name__ == "__main__":
    main()

# 2. Convolutional Neural Network

## 2.1 CNN Implementation (RGB)

In [ ]:
"""
Solving the CIFAR-10 classification task with a
Multi-Layer Perceptron NN structure.

Fine-Tuned Network
"""
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix


def unpickle(file):
    """
    Unpickles files produced with cPickle
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_cifar10():
    """
    Loads the CIFAR-10 dataset

    Output:
    X_train -> training images (RGB)
    y_train -> training images' labels
    X_test -> testing images (RGB)
    y_test -> testing images' labels
    """
    dict = unpickle("cifar-10-batches-py/data_batch_1")
    X_train = dict[b'data']
    y_train = np.array(dict[b'labels'], dtype=int)

    dict = unpickle("cifar-10-batches-py/data_batch_2")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_3")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_4")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_5")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/test_batch")
    X_test = dict[b'data']
    y_test = np.array(dict[b'labels'], dtype=int)

    # Reshape to (32, 32, 3)
    X_train = X_train.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype(int)
    X_test = X_test.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype(int)

    # Normalize data
    X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0

    # One-hot encode output, because allowing the model to assume a natural ordering 
    # between categories may result in poor performance or unexpected results
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)  #fit_transform learns the mean and variance of the data
    y_test_original = y_test  #maintain original labels
    y_test = lb.transform(y_test)  #tansform uses the same mean and var as above

    return X_train, y_train, X_test, y_test, y_test_original


def plot_model(history):
    # Plot accuracy
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title(f'Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_metric(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def print_examples(y_test, y_pred, X_test, labels):
    print('Printing examples of valid classification:')
    printed = 0
    fig = plt.figure(figsize=(4,4))
    for i in range (0, y_test.shape[0]):
        if y_test[i] == y_pred[i]:
            ax = fig.add_subplot(2, 2, printed + 1, xticks=[], yticks=[])
            ax.title.set_text(labels[y_pred[i]])
            ax.imshow((X_test[i, :]*255).astype(int))
            printed += 1
        if printed == 4:
            printed = 0
            break
    plt.show()
    print('Printing examples of false classification:')
    fig2 = plt.figure(figsize=(4,4))
    for i in range (0, y_test.shape[0]):
        if y_test[i] !=  y_pred[i]:
            ax2 = fig2.add_subplot(2, 2, printed + 1, xticks=[], yticks=[])
            ax2.title.set_text(labels[y_pred[i]])
            ax2.imshow((X_test[i, :]*255).astype(int))
            printed += 1
        if printed == 4:
            break
    plt.show()


def build_model():
    num_of_classes = 10
    input_shape = (32, 32, 3)

    # Create the model
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()

    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer, input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer=initializer))
    model.add(Dropout(0.2))
    model.add(Dense(num_of_classes, activation='softmax'))

    # Configure the model and start training
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.CategoricalAccuracy(), f1_metric])

    return model


def main():
    X_train, y_train, X_test, y_test, y_test_original = load_cifar10()
    model = build_model()
    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=40, batch_size=1000, verbose=1, validation_split=0.2)

    elapsed_time = time.time() - start_time
    print(f'Training time = {elapsed_time} second(s)')

    plot_model(history)

    # Test the model after training
    start_time = time.time()

    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    
    elapsed_time = time.time() - start_time
    print(f'Testing time = {elapsed_time} second(s)')
    
    # Plot confusion matrix
    prediction = model.predict(X_test) 
    y_pred = np.argmax(prediction, axis=1) #get predicted class
    labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    cm = confusion_matrix(y_test_original, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    fig, ax = plt.subplots(figsize=(15, 15))
    disp.plot(cmap=plt.cm.Blues, ax=ax)
    plt.show()

    print_examples(y_test_original, y_pred, X_test, labels)
    

if __name__ == "__main__":
    main()

## 2.2 CNN Implementation (Grayscale)

In [ ]:
"""
Solving the CIFAR-10 classification task with a
Multi-Layer Perceptron NN structure.

Fine-Tuned Network
"""
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
import time
import cv2
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix


def unpickle(file):
    """
    Unpickles files produced with cPickle
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_cifar10():
    """
    Loads the CIFAR-10 dataset

    Output:
    X_train -> training images (RGB)
    y_train -> training images' labels
    X_test -> testing images (RGB)
    y_test -> testing images' labels
    """
    dict = unpickle("cifar-10-batches-py/data_batch_1")
    X_train = dict[b'data']
    y_train = np.array(dict[b'labels'], dtype=int)

    dict = unpickle("cifar-10-batches-py/data_batch_2")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_3")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_4")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/data_batch_5")
    X_train = np.concatenate((X_train, dict[b'data']), axis=0)
    y_train = np.concatenate((y_train, np.array(dict[b'labels'], dtype=int)), axis=0)

    dict = unpickle("cifar-10-batches-py/test_batch")
    X_test = dict[b'data']
    y_test = np.array(dict[b'labels'], dtype=int)

    # Reshape to (32, 32, 3)
    X_train = X_train.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype(int)
    X_test = X_test.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype(int)

    # Convert to grayscale
    X_train = np.array([cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2GRAY) for image in X_train])
    X_test = np.array([cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_BGR2GRAY) for image in X_test])

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

    # Normalize data
    X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0

    # One-hot encode output, because allowing the model to assume a natural ordering 
    # between categories may result in poor performance or unexpected results
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)  #fit_transform learns the mean and variance of the data
    y_test_original = y_test  #maintain original labels
    y_test = lb.transform(y_test)  #tansform uses the same mean and var as above

    return X_train, y_train, X_test, y_test, y_test_original


def plot_model(history):
    # Plot accuracy
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title(f'Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_metric(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def build_model():
    num_of_classes = 10
    input_shape = (32, 32, 1)

    # Create the model
    model = Sequential()
    initializer = tf.keras.initializers.HeUniform()

    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer, input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', 
                            kernel_initializer=initializer))
    model.add(MaxPooling2D(2,2))

    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer=initializer))
    model.add(Dropout(0.2))
    model.add(Dense(num_of_classes, activation='softmax'))

    # Configure the model and start training
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.CategoricalAccuracy(), f1_metric])

    return model


def main():
    X_train, y_train, X_test, y_test, y_test_original = load_cifar10()
    model = build_model()
    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=40, batch_size=1000, verbose=1, validation_split=0.2)

    elapsed_time = time.time() - start_time
    print(f'Training time = {elapsed_time} second(s)')

    plot_model(history)

    # Test the model after training
    start_time = time.time()

    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    
    elapsed_time = time.time() - start_time
    print(f'Testing time = {elapsed_time} second(s)')
    """
    # Plot confusion matrix
    prediction = model.predict(X_test) 
    y_pred = np.argmax(prediction, axis=1) #get predicted class
    labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    cm = confusion_matrix(y_test_original, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    fig, ax = plt.subplots(figsize=(15, 15))
    disp.plot(cmap=plt.cm.Blues, ax=ax)
    plt.show()
    """

if __name__ == "__main__":
    main()